In [176]:
import torch
import pandas as pd
import numpy as np
from  sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
Data=pd.read_csv('data.csv')
display(Data.head())

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [177]:
Data.drop(columns=['id','Unnamed: 32'],inplace=True)

In [178]:
Data.shape

(569, 31)

Train,Test Split

In [179]:
X_Train,X_Test,y_Train,y_Test=train_test_split(Data.iloc[:,1:],Data.iloc[:,0],test_size=0.2,random_state=0)
X_Train


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
338,10.050,17.53,64.41,310.8,0.10070,0.07326,0.02511,0.01775,0.1890,0.06331,...,11.160,26.84,71.98,384.0,0.14020,0.14020,0.1055,0.06499,0.2894,0.07664
427,10.800,21.98,68.79,359.9,0.08801,0.05743,0.03614,0.01404,0.2016,0.05977,...,12.760,32.04,83.69,489.5,0.13030,0.16960,0.1927,0.07485,0.2965,0.07662
406,16.140,14.86,104.30,800.0,0.09495,0.08501,0.05500,0.04528,0.1735,0.05875,...,17.710,19.58,115.90,947.9,0.12060,0.17220,0.2310,0.11290,0.2778,0.07012
96,12.180,17.84,77.79,451.1,0.10450,0.07057,0.02490,0.02941,0.1900,0.06635,...,12.830,20.92,82.14,495.2,0.11400,0.09358,0.0498,0.05882,0.2227,0.07376
490,12.250,22.44,78.18,466.5,0.08192,0.05200,0.01714,0.01261,0.1544,0.05976,...,14.170,31.99,92.74,622.9,0.12560,0.18040,0.1230,0.06335,0.3100,0.08203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,18.810,19.98,120.90,1102.0,0.08923,0.05884,0.08020,0.05843,0.1550,0.04996,...,19.960,24.30,129.00,1236.0,0.12430,0.11600,0.2210,0.12940,0.2567,0.05737
9,12.460,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,...,15.090,40.68,97.65,711.4,0.18530,1.05800,1.1050,0.22100,0.4366,0.20750
359,9.436,18.32,59.82,278.6,0.10090,0.05956,0.02710,0.01406,0.1506,0.06959,...,12.020,25.02,75.79,439.6,0.13330,0.10490,0.1144,0.05052,0.2454,0.08136
192,9.720,18.22,60.73,288.1,0.06950,0.02344,0.00000,0.00000,0.1653,0.06447,...,9.968,20.83,62.25,303.8,0.07117,0.02729,0.0000,0.00000,0.1909,0.06559


Scaling

In [180]:
scalar=StandardScaler()
X_Train=scalar.fit_transform(X_Train)
X_Test=scalar.transform(X_Test)


In [181]:
X_Train
X_Test

array([[-0.20175604,  0.3290786 , -0.13086754, ...,  1.3893291 ,
         1.08203284,  1.54029664],
       [-0.25555773,  1.46763319, -0.31780437, ..., -0.83369364,
        -0.73131577, -0.87732522],
       [-0.02619262, -0.8407682 , -0.09175081, ..., -0.49483785,
        -1.22080864, -0.92115937],
       ...,
       [ 1.71811488,  0.09318356,  1.7286186 , ...,  1.57630515,
         0.20317063, -0.15406178],
       [ 1.18859296,  0.34352115,  1.19333694, ...,  0.56019755,
         0.26991966, -0.27320074],
       [ 0.26263752, -0.58080224,  0.28459338, ..., -0.19383705,
        -1.15564888,  0.11231497]], shape=(114, 30))

In [182]:
y_Train

338    B
427    B
406    B
96     B
490    B
      ..
277    M
9      M
359    B
192    B
559    B
Name: diagnosis, Length: 455, dtype: object

In [183]:
encoder=LabelEncoder()
y_Train=encoder.fit_transform(y_Train)
y_Test=encoder.transform(y_Test)


In [184]:
y_Train

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,

Numpy Array to Pytorch Tensor


In [185]:
X_Train_Tensor=torch.from_numpy(X_Train)
X_Test_Tensor=torch.from_numpy(X_Test)
y_Train_Tensor=torch.from_numpy(y_Train)
y_Test_Tensor=torch.from_numpy(y_Test)

In [186]:
X_Train_Tensor.shape

torch.Size([455, 30])

In [187]:
y_Train_Tensor.shape

torch.Size([455])

In [188]:
print(y_Train_Tensor)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
        0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,

Defining Model

In [189]:
class MySimpleNN:
    def __init__(self,x):
        self.weights=torch.rand(x.shape[1],1,requires_grad=True,dtype=torch.float64)
        self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)

    def forwardPass(self,X):
        z=torch.matmul(X,self.weights)+self.bias
        y_preds=torch.sigmoid(z)
        return y_preds
    def lossFuction(self,y_preds,y):
        epsilon=1e-7
        y_preds=torch.clamp(y_preds,epsilon,1-epsilon)
        loss=-(y_Train_Tensor*torch.log(y_preds)+(1-y_Train_Tensor)*torch.log(1-y_preds)).mean()
        return loss

Important Parameters

In [196]:
Learning_Rate=0.1
epochs=30

In [197]:
#Creating the Instance of the ModelClass
Model=MySimpleNN(X_Train_Tensor)
#Define Loop
for epoch in range(epochs):
    y_preds=Model.forwardPass(X_Train_Tensor)
    loss=Model.lossFuction(y_preds,y_Train_Tensor)
    loss.backward()
    with torch.no_grad():
        Model.weights-=Learning_Rate * Model.weights.grad
        Model.bias-=Learning_Rate * Model.bias.grad
    Model.weights.grad.zero_()
    Model.bias.grad.zero_()
    print(f"Epoch :{epoch+1}, and Loss:{loss.item()}")





Epoch :1, and Loss:3.6938850595708623
Epoch :2, and Loss:3.5829712515635173
Epoch :3, and Loss:3.46694214811099
Epoch :4, and Loss:3.346360294672677
Epoch :5, and Loss:3.2199022928622774
Epoch :6, and Loss:3.0833609967717273
Epoch :7, and Loss:2.943359187907149
Epoch :8, and Loss:2.794662124275495
Epoch :9, and Loss:2.6418834581022517
Epoch :10, and Loss:2.48140191673941
Epoch :11, and Loss:2.312119045215853
Epoch :12, and Loss:2.1447984769024666
Epoch :13, and Loss:1.9825111140219112
Epoch :14, and Loss:1.8264088103631215
Epoch :15, and Loss:1.6779220575817153
Epoch :16, and Loss:1.5369285169783082
Epoch :17, and Loss:1.3954561450245564
Epoch :18, and Loss:1.265768445280716
Epoch :19, and Loss:1.1523763060630052
Epoch :20, and Loss:1.0566365012047658
Epoch :21, and Loss:0.9790094838470735
Epoch :22, and Loss:0.9187487450679414
Epoch :23, and Loss:0.873813765102235
Epoch :24, and Loss:0.8412311970352099
Epoch :25, and Loss:0.8177807897133158
Epoch :26, and Loss:0.8006311869887551
Epoch

In [198]:
Model.bias

tensor([-0.1786], dtype=torch.float64, requires_grad=True)

Model Evaluation